<a href="https://colab.research.google.com/github/senthilsriniappyhub/TTS/blob/main/Colab/StyleTTS2_Finetune_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages and download models

In [4]:
%%shell
git clone https://github.com/yl4579/StyleTTS2.git
cd StyleTTS2
pip install SoundFile torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
sudo apt-get install espeak-ng
git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LibriTTS
mv StyleTTS2-LibriTTS/Models .

Cloning into 'StyleTTS2'...
remote: Enumerating objects: 372, done.
remote: Total 372 (delta 0), reused 0 (delta 0), pack-reused 372 (from 1)
Receiving objects: 100% (372/372), 133.98 MiB | 18.38 MiB/s, done.
Resolving deltas: 100% (199/199), done.
Updating files: 100% (48/48), done.
  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-g_wko2qy
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-g_wko2qy
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
 

### Download dataset (LJSpeech, 200 samples, ~15 minutes of data)

You can definitely do it with fewer samples. This is just a proof of concept with 200 smaples.

In [5]:
%cd StyleTTS2
!rm -rf Data

/content/StyleTTS2


In [7]:
#!gdown --id 1vqz26D3yn7OXS2vbfYxfSnpLS6m6tOFP
!gdown --id 1UO_8yjJtPXWIYk6vXa1rOKhrqjpwEqZ8
!unzip Data.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1UO_8yjJtPXWIYk6vXa1rOKhrqjpwEqZ8
From (redirected): https://drive.google.com/uc?id=1UO_8yjJtPXWIYk6vXa1rOKhrqjpwEqZ8&confirm=t&uuid=927efed3-b417-4bdf-86a2-b10c98f65ade
To: /content/StyleTTS2/Data.zip
100% 49.1M/49.1M [00:01<00:00, 40.5MB/s]
Archive:  Data.zip
replace Data/LJ001-0048.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
input_file_name = "200_sample.txt"
output_file_name = "phonemized_200.txt"
# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

def text_to_phonemes(text):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    return ps[0]

file_out = open(output_file_name, 'w')

with open(input_file_name, 'r', encoding='utf-8') as f:
    for line in f:
      #print(line)
      wave_file, text = line.split('|', 1)
      wave_file_name = wave_file + '.wav'
      #print(wave_file_name, text)
      phonemized = text_to_phonemes(text)
      #print(phonemized)
      #file_out.write(wave_file_name+ '.wav' + '|' + phonemized +'|'+'0'+'\n')
      file_out.write(f"{wave_file_name}|{phonemized}|{0}\n")
file_out.close()

In [9]:
!head -n 150 phonemized_200.txt > Data/train_list.txt
!tail -n 50 phonemized_200.txt > Data/val_list.txt
!head -n 150 phonemized_200.txt > Data/OOD_texts.txt
!head -40 Data/OOD_texts.txt
!head -20 Data/val_list.txt
!head -20 Data/OOD_texts.txt

LJ001-0001.wav|pɹˈɪntɪŋ, ɪnðɪ ˈoʊnli sˈɛns wɪð wˌɪtʃ wiː ɑːɹ æt pɹˈɛzənt kənsˈɜːnd, dˈɪfɚz fɹʌm mˈoʊst ɪf nˌɑːt fɹʌm ˈɔːl ðɪ ˈɑːɹts ænd kɹˈæfts ɹˌɛpɹᵻzˈɛntᵻd ɪnðɪ ɛksɪbˈɪʃən pɹˈɪntɪŋ, ɪnðɪ ˈoʊnli sˈɛns wɪð wˌɪtʃ wiː ɑːɹ æt pɹˈɛzənt kənsˈɜːnd, dˈɪfɚz fɹʌm mˈoʊst ɪf nˌɑːt fɹʌm ˈɔːl ðɪ ˈɑːɹts ænd kɹˈæfts ɹˌɛpɹᵻzˈɛntᵻd ɪnðɪ ɛksɪbˈɪʃən |0
LJ001-0002.wav|ɪn bˌiːɪŋ kəmpˈæɹətˌɪvli mˈɑːdɚn.ɪn bˌiːɪŋ kəmpˈæɹətˌɪvli mˈɑːdɚn. |0
LJ001-0003.wav|fɔːɹ ɔːlðˈoʊ ðə tʃaɪnˈiːz tˈʊk ɪmpɹˈɛʃənz fɹʌm wˈʊd blˈɑːks ɛŋɡɹˈeɪvd ɪn ɹᵻlˈiːf fɔːɹ sˈɛntʃɚɹiz bᵻfˌoːɹ ðə wˈʊdkʌɾɚz ʌvðə nˈɛðɜːləndz, baɪ ɐ sˈɪmɪlɚ pɹˈɑːsɛs fɔːɹ ɔːlðˈoʊ ðə tʃaɪnˈiːz tˈʊk ɪmpɹˈɛʃənz fɹʌm wˈʊd blˈɑːks ɛŋɡɹˈeɪvd ɪn ɹᵻlˈiːf fɔːɹ sˈɛntʃɚɹiz bᵻfˌoːɹ ðə wˈʊdkʌɾɚz ʌvðə nˈɛðɜːləndz, baɪ ɐ sˈɪmɪlɚ pɹˈɑːsɛs |0
LJ001-0004.wav|pɹədˈuːst ðə blˈɑːk bˈʊks, wˌɪtʃ wɜː ðɪ ɪmˈiːdɪət pɹˈɛdᵻsˌɛsɚz ʌvðə tɹˈuː pɹˈɪntᵻd bˈʊk,pɹədˈuːst ðə blˈɑːk bˈʊks, wˌɪtʃ wɜː ðɪ ɪmˈiːdɪət pɹˈɛdᵻsˌɛsɚz ʌvðə tɹˈuː pɹˈɪntᵻd bˈʊk, |0
LJ001-0005.wav|ðɪ ɪnvˈɛnʃən ʌv mˈuːvəbəl mˈɛɾəl 

### Change the finetuning config

Depending on the GPU you got, you may want to change the bacth size, max audio length, epiochs and so on.

In [10]:
config_path = "Configs/config_ft.yml"

import yaml
config = yaml.safe_load(open(config_path))
!cat Configs/config_ft.yml
!head Data/train_list.txt
!head 200_sample.txt

log_dir: "Models/LJSpeech"
save_freq: 5
log_interval: 10
device: "cuda"
epochs: 50 # number of finetuning epoch (1 hour of data)
batch_size: 8
max_len: 400 # maximum number of frames
pretrained_model: "Models/LibriTTS/epochs_2nd_00020.pth"
second_stage_load_pretrained: true # set to true if the pre-trained model is for 2nd stage
load_only_params: true # set to true if do not want to load epoch numbers and optimizer parameters

F0_path: "Utils/JDC/bst.t7"
ASR_config: "Utils/ASR/config.yml"
ASR_path: "Utils/ASR/epoch_00080.pth"
PLBERT_dir: 'Utils/PLBERT/'

data_params:
  train_data: "Data/train_list.txt"
  val_data: "Data/val_list.txt"
  root_path: "/local/LJSpeech-1.1/wavs"
  OOD_data: "Data/OOD_texts.txt"
  min_length: 50 # sample until texts with this size are obtained for OOD texts

preprocess_params:
  sr: 24000
  spect_params:
    n_fft: 2048
    win_length: 1200
    hop_length: 300

model_params:
  multispeaker: true

  dim_in: 64 
  hidden_dim: 512
  max_conv_dim: 512
  n_layer: 

In [11]:
config['data_params']['root_path'] = "Data/"

config['batch_size'] = 2 # not enough RAM
config['save_freq'] = 1
config['max_len'] = 100 # not enough RAM
config['epochs'] = 50
config['loss_params']['diff_epoch'] = 10
config['loss_params']['joint_epoch'] = 110 # we do not do SLM adversarial training due to not enough RAM

with open(config_path, 'w') as outfile:
  yaml.dump(config, outfile, default_flow_style=True)

### Start finetuning


In [12]:
!pwd
!python train_finetune.py --config_path ./Configs/config_ft.yml

/content/StyleTTS2
2025-03-24 13:17:10.206535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742822230.437746    4086 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742822230.496812    4086 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 13:17:11.003677: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/StyleTTS2/train_finetune.py", line 707, in <mo

### Test the model quality

Note that this mainly serves as a proof of concept due to RAM limitation of free Colab instances. A lot of settings are suboptimal. In the future when DDP works for train_second.py, we will also add mixed precision finetuning to save time and RAM. You can also add SLM adversarial training run if you have paid Colab services (such as A100 with 40G of RAM).

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install munch

In [5]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(path):
    wave, sr = librosa.load(path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:
        audio = librosa.resample(audio, sr, 24000)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("Models/LJSpeech/config_ft.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model_params = recursive_munch(config['model_params'])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

ModuleNotFoundError: No module named 'models'

In [ ]:
files = [f for f in os.listdir("Models/LJSpeech/") if f.endswith('.pth')]
sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

In [ ]:
params_whole = torch.load("Models/LJSpeech/" + sorted_files[-1], map_location='cpu')
params = params_whole['net']

<ipython-input-12-febab44deea8>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params_whole = torch.load("Models/LJSpeech/" + sorted_files[-1], map_location='cpu')


In [ ]:
for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

bert loaded
bert_encoder loaded
predictor loaded
decoder loaded
text_encoder loaded
predictor_encoder loaded
style_encoder loaded
diffusion loaded
text_aligner loaded
pitch_extractor loaded
mpd loaded
msd loaded
wd loaded


In [ ]:
from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

In [ ]:
sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

In [ ]:
def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    #ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    print(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en,
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr,
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later

### Synthesize speech

In [ ]:
text = '''Maltby and Company would issue warrants on them deliverable to the importer, and the goods were then passed to be stored in neighboring warehouses.
'''
text = "text to speech synthesis from Machine."
# text = "Speech synthesis is the artificial production of human speech."

In [ ]:
# get a random reference in the training set, note that it doesn't matter which one you use
path = "Data/LJ001-0110.wav"
# this style vector ref_s can be saved as a parameter together with the model weights
ref_s = compute_style(path)

In [ ]:
start = time.time()
wav = inference(text, ref_s, alpha=0, beta=0, diffusion_steps=10, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
import IPython.display as ipd
display(ipd.Audio(wav, rate=24000, normalize=False))

display(ipd.Audio(path, rate=24000, normalize=False))

tˈɛkst tə spˈiːtʃ sˈɪnθəsˌɪs fɹʌm məʃˈiːn. 
RTF = 0.121883
